In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import time
import numpy as np


In [30]:
data = pd.read_csv("H:\My Drive\Working Files CS\Analytics\WI Club Nights\WI Club Night - Pivot table.csv")
display(data.head(3))

,Event Date,Topic,Attended,User Name (Original Name),First Name,Last Name,Email,City,Country/Region,State/Province,Phone,Registration Time,Approval Status,Join Time,Leave Time,Time in Session (minutes),Is Guest,Country/Region Name,Source Name
0,"Apr 05, 2022",Renewable Energy,Yes,Dhan .,Dhan,.,junglenutcracker@gmail.com,NaN,SG,Other,NaN,"Apr 05, 2022 09:55:55",approved,"Apr 05, 2022 17:48:39","Apr 05, 2022 19:21:03",93,Yes,Singapore,Email
1,"Apr 05, 2022",Renewable Energy,Yes,Joseph Bampton,Joseph,Bampton,bamptonj01@icloud.com,NaN,AU,NaN,NaN,"Apr 04, 2022 13:54:02",approved,"Apr 05, 2022 18:02:11","Apr 05, 2022 18:09:37",8,Yes,Australia,NaN
2,"Apr 05, 2022",Renewable Energy,Yes,Sophie Bayar,Sophie,Bayar,sophie.bayars@gmail.com,Sydney,AU,NSW,451468778,"Apr 04, 2022 15:07:34",approved,"Apr 05, 2022 17:29:05","Apr 05, 2022 18:05:27",37,Yes,Australia,Email


In [3]:
display(data.shape)

(314, 19)

In [4]:
# Convert time into the time format
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%b %d, %Y')
data['Registration Time'] = pd.to_datetime(data['Registration Time'], format='%b %d, %Y %H:%M:%S')
data['Join Time'] = pd.to_datetime(data['Join Time'], format='%b %d, %Y %H:%M:%S', errors='coerce')
data['Leave Time'] = pd.to_datetime(data['Leave Time'], format='%b %d, %Y %H:%M:%S', errors='coerce')

In [5]:
# Convert Time in Session (minutes) into the integer format
data['Time in Session (minutes)'] = data['Time in Session (minutes)'].fillna(0)
data['Time in Session (minutes)'] = data['Time in Session (minutes)'].apply(lambda x: 0 if x == '--' else x)

data['Time in Session (minutes)'] = data['Time in Session (minutes)'].astype('int64')

In [6]:
data['Attended (for more than 5 mins)'] = data['Time in Session (minutes)'].apply(lambda x: 1 if x >= 5 else 0)
data['Attended (for less than 5 mins)'] = data['Time in Session (minutes)'].apply(lambda x: 1 if 0 < x < 5 else 0)

#data[data['Time in Session (minutes)'] > 5]

In [7]:
data['Status'] = data['Attended']
data['Status'] = data['Status'].apply(lambda x: 'Attended' if x == 'Yes' else 'Registered')
data['Attended'] = data['Attended'].apply(lambda x: 1 if x == 'Yes' else 0)
data['Approval Status'] = data['Approval Status'].apply(lambda x: 1 if x == 'approved' else 0)
data.rename(columns = {'Approval Status':'Registered'}, inplace = True)
data = data.drop('Is Guest', axis=1)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Event Date                       314 non-null    datetime64[ns]
 1   Topic                            314 non-null    object        
 2   Attended                         314 non-null    int64         
 3   User Name (Original Name)        314 non-null    object        
 4   First Name                       314 non-null    object        
 5   Last Name                        314 non-null    object        
 6   Email                            314 non-null    object        
 7   City                             137 non-null    object        
 8   Country/Region                   230 non-null    object        
 9   State/Province                   170 non-null    object        
 10  Phone                            105 non-null    object       

In [9]:
null_data = data.isnull().sum()
display(null_data[null_data > 0])

City                   177
Country/Region          84
State/Province         144
Phone                  209
Join Time              192
Leave Time             192
Country/Region Name     84
Source Name             34
dtype: int64

In [10]:
# The number of attendees and their average watch time

reg_att_data = data.groupby(['Event Date'],as_index=False)[['Registered', 'Attended (for more than 5 mins)', 'Attended (for less than 5 mins)']].sum()
display(reg_att_data.round(2))

,Event Date,Registered,Attended (for more than 5 mins),Attended (for less than 5 mins)
0,2022-04-05,99,37,5
1,2022-04-12,65,25,5
2,2022-04-25,77,16,7
3,2022-05-03,36,9,0
4,2022-05-10,37,16,2


In [11]:


barchart = go.Figure(data=[
    go.Bar(
        name='Registered', 
        x=reg_att_data['Event Date'], 
        y=reg_att_data['Registered'],
        text=reg_att_data['Registered'],
        offsetgroup=0,
#         marker_color='rgb(54, 28, 84)'
    ),
    
    go.Bar(
        name='Attended (for less than 5 mins)', 
        x=reg_att_data['Event Date'], 
        y=reg_att_data['Attended (for less than 5 mins)'],
        text=reg_att_data['Attended (for less than 5 mins)'],
        offsetgroup=1,
#         marker_color='rgb(213, 189, 147)'
    ),
    
    go.Bar(
        name='Attended (for more than 5 mins)', 
        x=reg_att_data['Event Date'], 
        y=reg_att_data['Attended (for more than 5 mins)'],
        text=reg_att_data['Attended (for more than 5 mins)'],
        offsetgroup=1,
        base=reg_att_data['Attended (for less than 5 mins)'],
#         marker_color='rgb(118, 92, 46)'
    )
])

# Change the bar mode
barchart.update_layout(
        title="WI Club Investor Night - Registration vs Attendance",
        yaxis_title="Number of People",
        width=1000,
        height=600,
        legend=dict(
        x=0.7,
        y=0.96,
        bgcolor='rgba(255, 255, 255, 0)', # transparent legend background
        bordercolor='rgba(255, 255, 255, 0)' # transparent legend borders
    )
    )
barchart.show()

In [12]:
# pip install -U kaleido

In [13]:
t = time.localtime()
timestamp = time.strftime('%Y-%m-%d', t)
barchart_file_name = (' - ' + timestamp)
location_windows = "H:\My Drive\Working Files CS\Analytics\WI Club Nights\Registrations vs Attendees Barchart"

pio.write_image(barchart, (location_windows + barchart_file_name + '.png'), format='png', width=1000, height=600)


In [14]:

reg_unique = data['Email'].nunique()
print('{} unique registrations across all events'.format(reg_unique))

mask = data[data['Attended'] == 1]
att_unique = mask['Email'].nunique()
print('{} unique attendees across all events'.format(att_unique))


222 unique registrations across all events
100 unique attendees across all events


In [15]:
# The number of attendees and their average watch time

pie_data = data.groupby(['Email'],as_index=False)[['Registered', 'Attended (for more than 5 mins)', 'Attended (for less than 5 mins)']].sum()
display(pie_data.sort_values('Attended (for more than 5 mins)', ascending=False))

,Email,Registered,Attended (for more than 5 mins),Attended (for less than 5 mins)
90,hodgkinsons1@bigpond.com,4,4,0
91,homehelper@protonmail.com,4,3,0
5,Dommccormick12@gmail.com,4,3,0
212,trevor@riodefi.com,4,3,1
79,gerard.kevin@firstreliancecapital.com,4,3,0
...,...,...,...,...
96,jeff.lock@bigpond.com,4,0,0
97,jeff.mansfield@addisons.com,1,0,0
98,jesse.loong@oyob.com.au,1,0,0
101,jm.floch@flochinternational.com,1,0,0


In [16]:
pie_data = pd.DataFrame(pie_data)
display(pie_data.nunique())

Email                              222
Registered                           5
Attended (for more than 5 mins)      5
Attended (for less than 5 mins)      2
dtype: int64

In [34]:
display(pie_data.head(3))

,Email,Registered,Attended (for more than 5 mins),Attended (for less than 5 mins),Attended
0,Aleks@belay.com.au,1,1,0,once
1,Angela@agfunder.com,3,0,0,0
2,Chris@montu.com.au,1,0,0,0


In [17]:
def get_attendance(value):
    if value == 1:
        return 'once'
    elif value == 2:
        return 'twice'
    elif value == 3:
        return 'three times'
    elif value == 4:
        return 'four times'
    elif value > 5:
        return 'five times or more'
    else: 
        return 0

pie_data['Attended'] = pie_data['Attended (for more than 5 mins)'].apply(get_attendance)

In [50]:
mask = pie_data[pie_data['Attended (for more than 5 mins)'] > 0]
pie_data2 = mask['Attended'].value_counts(normalize=True)
piechart = px.pie(
    pie_data2,
    values='Attended',
    names=pie_data2.index,
    width=1000,
    height=600,
    hole=0.3,
    title='How many times people came to WI Club Nights (Overall)'
)
piechart.update_traces(textinfo='percent+label')
piechart.show()


# display(pie_data.sort_values('Attended (for more than 5 mins)', ascending=False))

In [19]:
piechart_file_name = (' - ' + timestamp)
location_windows = "H:\My Drive\Working Files CS\Analytics\WI Club Nights\Repeat Attendees Piechart"

pio.write_image(piechart, (location_windows + piechart_file_name + '.png'), format='png', width=1000, height=600)


In [20]:
data = data.merge(
    pie_data,
    on='Email',
    how='left')

In [21]:
data = data.drop(['Attended (for more than 5 mins)_y', 'Attended (for less than 5 mins)_y'], axis=1)
data = data.rename(columns={"Attended (for more than 5 mins)_x": "Attended (for more than 5 mins)", 
                            "Attended (for less than 5 mins)_x": "Attended (for less than 5 mins)", 
                            "Registered_y": "Number of visits", "Attended_y": "Visited", "Attended_x": "Attended"})

In [22]:
conditions = [
    data['Event Date'].eq('2022-04-05') & data['Visited'].isin(['twice', 'three times', 'four times']),
    data['Event Date'].eq('2022-04-12') & data['Visited'].isin(['three times', 'four times']),
    data['Event Date'].eq('2022-04-25') & data['Visited'].eq('three times'),
    data['Event Date'].eq('2022-04-25') & data['Visited'].eq('four times')
]

choices = ['once', 'twice', 'twice', 'three times']

data['Visited'] = np.select(conditions, choices, default=data['Visited'])

In [23]:
mask2 = data[data['Attended (for more than 5 mins)'] > 0]

visits_data = mask2.groupby(
    ['Event Date', 'Visited'], as_index=False
    )[['Attended (for more than 5 mins)']].sum()

visits_data = pd.DataFrame(visits_data)

In [24]:
barchart2 = px.bar(
    data_frame=visits_data,
    x="Event Date",
    y="Attended (for more than 5 mins)",
    color="Visited",			# цвета для столбиков
    # opacity=0.9,
    # orientation="v",
    barmode='group',				# 'overlay' / 'relative'
    category_orders={'Visited':
    ['once','twice','three times',
     'four times']},			# можно задать порядок столбцов
    # labels={"Attended (for more than 5 mins)":"Attended"},				# можно прописать названия
    # title='Repeat viewers by event',
    width=1000,
    height=600,
    text='Attended (for more than 5 mins)',
    # template='none'
)

barchart2.update_layout(
        title="Repeat viewers by event",
        title_x=0.5,
        title_y=0.965,
        title_font_size=22,
        yaxis_title="Number of People",
        legend=dict(
        x=0.84,
        y=0.95,
        bgcolor='rgba(255, 255, 255, 0)', # transparent legend background
        bordercolor='rgba(255, 255, 255, 0)' # transparent legend borders
    )
    )

barchart2.show()


In [51]:
pie_data_all = mask
pie_data_12_04_2022 = mask2[mask2['Event Date'] == '2022-04-12']
pie_data_10_05_2022 = mask2[mask2['Event Date'] == '2022-05-10']
# pie_data_17_05_2022 = mask2[mask2['Event Date'] == '2022-05-17']

In [26]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

barchart_2022_04_12 = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],
    subplot_titles=("On 12/04/2022","Overall")
    )


barchart_2022_04_12.add_trace(go.Pie(
    values=pie_data_12_04_2022['Visited'].value_counts(),
    # names=pie_data_12_04_2022.index,
    hole=0.3
    ),
              row=1, col=1)

barchart_2022_04_12.add_trace(go.Pie(
    values=pie_data_all['Attended'].value_counts(),
    # names=pie_data2.index,
    hole=0.3
    ),
              row=1, col=2)

barchart_2022_04_12.update_layout(
    width=1000, height=600,
    showlegend=False,
    title_text="Repeat Attendees",
    title_x=0.5,
    title_y=0.965,
    title_font_size=22,
    )

# newnames = {'0':'once', '1': 'twice', '2':'three times', '3':'four times'}
# fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))

barchart_2022_04_12.show()

In [27]:
# piechart2_file_name = (' - Repeat Attendees 2022-04-12 - ' + timestamp)
# location_windows = "H:\My Drive\Working Files CS\Analytics\WI Club Nights\WI Club Night"

# pio.write_image(barchart_2022_04_12, (location_windows + piechart2_file_name + '.png'), format='png', width=1000, height=600)

In [52]:
barchart_2022_05_10 = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],
    subplot_titles=("On 10/05/2022 (Property and Funds)","Overall")
    )


barchart_2022_05_10.add_trace(go.Pie(
    values=pie_data_10_05_2022['Visited'].value_counts(),
    # names=pie_data_12_04_2022.index,
    hole=0.3
    ),
              row=1, col=1)

barchart_2022_05_10.add_trace(go.Pie(
    values=pie_data_all['Attended'].value_counts(),
    # names=pie_data2.index,
    hole=0.3
    ),
              row=1, col=2)

barchart_2022_05_10.update_layout(
    width=1000, height=600,
    showlegend=False,
    title_text="Repeat Attendees",
    title_x=0.5,
    title_y=0.965,
    title_font_size=22,
    )

barchart_2022_05_10.show()